In [23]:
import numpy as np 
import pandas as pd

from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import gc

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional

import tensorflow as tf
import keras.backend as K



## reload scripts before executing them
%load_ext autoreload
%autoreload 2

from dataset.reduce_memory import reduce_mem_usage

from dataset.create_windows import create_windows
from dataset.create_submission import create_submission_csv

from evaluation.rmsse_loss import root_mean_squared_scaled_error

from train.cross_validate import cross_validate

from models.lstm import lstm_model


from evaluation.wrmsse_loss import WRMSSE_Loss

import wandb
from wandb.keras import WandbCallback

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
## Init Weights and Biases

wandb.init(sync_tensorboard=True, entity="whydoihavetoregister", project="m5_accuracy_forecasting")


W&B Run: https://app.wandb.ai/whydoihavetoregister/m5_accuracy_forecasting/runs/3uvp79ev

In [3]:
# read data

data_dir = 'data/'

train_sales = pd.read_csv(data_dir + 'sales_train_validation.csv')
#sell_prices = pd.read_csv(data_dir + 'sell_prices.csv')
calendar = pd.read_csv(data_dir + 'calendar.csv')
submission_file = pd.read_csv(data_dir + 'sample_submission.csv')


In [4]:
train_sales = reduce_mem_usage(train_sales) # takes about 4mins

Mem. usage decreased to 95.00 Mb (78.7% reduction)


In [5]:
# create training data
sales = train_sales.drop(["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"], axis=1).T

# normalize
scaler = MinMaxScaler()
scaler.fit(sales)
sales = scaler.transform(sales)
sales = pd.DataFrame(sales)

In [6]:
file_pass = './'

# Load S and W weights for WRMSSE calcualtions:
sw_df = pd.read_pickle(file_pass+'sw_df.pkl')
S = sw_df.s.values
W = sw_df.w.values
SW = sw_df.sw.values

# Load roll up matrix to calcualte aggreagates:
roll_mat_df = pd.read_pickle(file_pass+'roll_mat_df.pkl')
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)
del roll_mat_df

In [7]:
# create X and y

timesteps = 28
prediction_steps = 1

X,y = create_windows(sales, timesteps,prediction_steps)

In [13]:
WRMSSE_Loss = WRMSSE_Loss(SW,roll_mat_csr)
wrmsse_loss_method = WRMSSE_Loss.wrmsse_loss

In [14]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor = 0.1, patience = 2 , verbose = 1, mode= 'min', min_lr = 0.000001)


In [15]:
n_features = X.shape[2]


In [24]:
lstm_model = lstm_model(timesteps,n_features,loss=wrmsse_loss_method)


In [25]:
cross_validate(lstm_model,X,y,batch_size=32,epochs = 15, number_of_folds = 5,callbacks =[reduce_lr,WandbCallback()])

Epoch 1/15
315/315 [==============================] - 20s 63ms/step - loss: 0.1062 - mse: 0.0144
Epoch 2/15
315/315 [==============================] - 11s 36ms/step - loss: 0.0591 - mse: 0.0130
Epoch 3/15
315/315 [==============================] - 12s 39ms/step - loss: 0.0495 - mse: 0.0123
Epoch 4/15
315/315 [==============================] - 12s 39ms/step - loss: 0.0452 - mse: 0.0117
Epoch 5/15
315/315 [==============================] - 12s 38ms/step - loss: 0.0444 - mse: 0.0113
Epoch 6/15
315/315 [==============================] - 11s 34ms/step - loss: 0.0421 - mse: 0.0110
Epoch 7/15
315/315 [==============================] - 11s 34ms/step - loss: 0.0400 - mse: 0.0108
Epoch 8/15
315/315 [==============================] - 11s 34ms/step - loss: 0.0395 - mse: 0.0106
Epoch 9/15
315/315 [==============================] - 11s 35ms/step - loss: 0.0384 - mse: 0.0104
Epoch 10/15
315/315 [==============================] - 11s 34ms/step - loss: 0.0379 - mse: 0.0102
Epoch 11/15
315/315 [========

([0.05609799484918072,
  0.054176644250086156,
  0.052902900346904806,
  0.05797216827702371,
  0.06454920930087946],
 0.05713978340481497,
 0.00408433964738724)